<a href="https://colab.research.google.com/github/cantones1/Coursera_Capstone/blob/master/DL0321EN_4_1_Comparing_Models_py_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-10-12 02:06:17--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  28.8MB/s    in 8.9s    

2020-10-12 02:06:27 (28.1 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip concrete_data_week4.zip

Streaming output truncated to the last 5000 lines.
  inflating: concrete_data_week4/train/negative/08835.jpg  
  inflating: __MACOSX/concrete_data_week4/train/._negative  


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:
valid_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [8]:
modelV = Sequential()

modelV.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

modelV.add(Dense(num_classes, activation='softmax'))

58892288/58889256 [==============================] - 0s 0us/step


In [9]:
modelV.layers[0].trainable = False

modelV.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)
num_epochs = 2

In [10]:
fit_history_V = modelV.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 167s 556ms/step - loss: 0.0894 - accuracy: 0.9700 - val_loss: 0.0241 - val_accuracy: 0.9947
Epoch 2/2
301/301 [==============================] - 167s 555ms/step - loss: 0.0193 - accuracy: 0.9957 - val_loss: 0.0153 - val_accuracy: 0.9967


<a id="item43"></a>

In [21]:
modelV.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
modelR=keras.models.load_model('/content/drive/My Drive/Colab Notebooks/classifier_resnet_model.h5')

In [13]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [14]:
# Evaluate ResNet on test data
resultsR = modelR.evaluate(test_generator)
print(f"ResNet evaluation on test data: loss={resultsR[0]}, accuracy={resultsR[1]}")

16/16 [==============================] - 2s 117ms/step - loss: 0.0021 - accuracy: 1.0000
ResNet evaluation on test data: loss=0.0020538847893476486, accuracy=1.0


In [15]:
# Evaluate VGG16 on test data
resultsV = modelV.evaluate(test_generator)
print(f"VGG16 evaluation on test data: loss={resultsV[0]}, accuracy={resultsV[1]}")

16/16 [==============================] - 3s 180ms/step - loss: 0.0126 - accuracy: 0.9980
VGG16 evaluation on test data: loss=0.0126021858304739, accuracy=0.9980000257492065


In [16]:
print(f"ResNet evaluation on test data: loss={resultsR[0]}, accuracy={resultsR[1]}")
print(f"VGG16 evaluation on test data: loss={resultsV[0]}, accuracy={resultsV[1]}")

ResNet evaluation on test data: loss=0.0020538847893476486, accuracy=1.0
VGG16 evaluation on test data: loss=0.0126021858304739, accuracy=0.9980000257492065


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [17]:
modelR_predictions=modelR.predict_generator(test_generator[0])
modelV_predictions=modelV.predict_generator(test_generator[0])

Instructions for updating:
Please use Model.predict, which supports generators.


In [18]:
for index in range(5):
  if modelR_predictions.round()[index][0]:
    print("Negative")
  else:
    print("Positive")



Negative
Negative
Negative
Negative
Negative


In [19]:
modelV_predictions.round()
for index in range(5):
  if modelV_predictions.round()[index][0]:
    print("Negative")
  else:
    print("Positive")

Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).